In [1]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [2]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng1train.csv', encoding="latin-1")
df_train.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,dhonikeepstheglove watch sports minister kire...,1,NONE,NONE
1,hasoc_en_2,@politico no we should remember very clearly t...,0,HATE,TIN
2,hasoc_en_3,@cricketworldcup guess who would be the winner...,1,NONE,NONE
3,hasoc_en_4,corbyn is too politically intellectual for bor...,1,NONE,NONE
4,hasoc_en_5,all the best to teamindia for another swimming...,1,NONE,NONE


In [3]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_1'],random_state=140) 

In [4]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [5]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng1test.csv', encoding="latin-1") 
test_y = encoder.fit_transform(df_test['task_1']) #encoding the label of testing data

In [6]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)  #get count vector features for training data
xvalid_count =  count_vect.transform(valid_x)  #get count vector features for validation data
x_count =  count_vect.transform(df_test['text'])  #get count vector features for testing data

In [7]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [8]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc(),x_count.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.22      0.72      0.34       183
           1       0.94      0.64      0.76      1280

    accuracy                           0.65      1463
   macro avg       0.58      0.68      0.55      1463
weighted avg       0.85      0.65      0.71      1463

classification report for testing
              precision    recall  f1-score   support

           0       0.41      0.74      0.53       159
           1       0.95      0.83      0.89       994

    accuracy                           0.82      1153
   macro avg       0.68      0.79      0.71      1153
weighted avg       0.88      0.82      0.84      1153

